In [1]:
from docplex.mp.model import Model
import pandas as pd

### 数据导入模块

In [2]:
f=r'data_v1.xlsx'
data_ferry=pd.read_excel(f,sheet_name="Sheet2-ferry")
data_port=pd.read_excel(f,sheet_name="Sheet3-1")

In [3]:
data_ferry.head()

,f,hf,Cf,gf_enroute,gf_inport
0,small1,1,100.0,400.0,160.0
1,small2,3,100.0,400.0,160.0
2,large,5,200.0,600.0,240.0
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN


### 参数设定

In [4]:
m = 3 #轮船数
n = 5 #港口数
q = 115 #时刻数

In [5]:
lam = 1 # 轮船成本系数
niu = 10 # 客户成本系数

#### 下标与集合

In [6]:
F = range(0,m) #轮船集合
P = range(0,n) #港口集合
Q = range(0,q) #时刻集合

In [7]:
#参数list初始化
cap = [0] * m #轮船集合
home_port = [0] * m # 轮船的母港
berth = [0] * n #每个港口的可容纳的轮船数

In [8]:
w=[[0 for k in P] for f in F]
i=0
j=0
for index,row in data_port.iterrows():
    port,time=row
    port=int(port[-1])
    j+=1
    w[i][port-1]=int(time/10)
    if j%5==0:
        i+=1

In [9]:
w

[[3, 2, 2, 2, 99], [3, 2, 2, 2, 99], [3, 2, 2, 2, 4]]

In [10]:
for i in F:
    cap[i] = data_ferry["Cf"][i]
    home_port[i] = data_ferry['hf'][i]-1

for i in P:
    berth[i] = 2

#### 节点定义

In [11]:
data_demand=pd.read_excel(f,sheet_name="Sheet5-1")
# 列更名+删除无用行
data_demand.columns=['port','cus_port','series','volume','arrival_time']
data_demand=data_demand.drop(0,axis=0)

In [12]:
# 根据sheet5生成所有node
V=[[] for i in P] # 对每个港口都生成node   
# V中node的格式：(初始port,时刻上的序号)
for i in P:
    for j in Q:        
        node=(i,j) 
        V[i].append(node)

In [13]:
VV=[]
for v in V:
    for i in v:
        VV.append(i)

In [14]:
# 对demand列表循环，将客户需求、客户目的地、客户到达时间更新
D=[{} for t in P]
arr_time=[{} for t in P]
for t in P:
    for v in VV:
        D[t][v]=0
        arr_time[t][v]=q
for index,row in data_demand.iterrows():  
    i, a, j, b, c = row
    D[a-1][(i-1,j-1)]=b
    arr_time[a-1][(i-1,j-1)]=c

In [15]:
D[0]

{(0, 0): 0,
 (0, 1): 0,
 (0, 2): 0,
 (0, 3): 0,
 (0, 4): 0,
 (0, 5): 0,
 (0, 6): 0,
 (0, 7): 0,
 (0, 8): 0,
 (0, 9): 0,
 (0, 10): 0,
 (0, 11): 0,
 (0, 12): 0,
 (0, 13): 0,
 (0, 14): 0,
 (0, 15): 0,
 (0, 16): 0,
 (0, 17): 0,
 (0, 18): 0,
 (0, 19): 0,
 (0, 20): 0,
 (0, 21): 0,
 (0, 22): 0,
 (0, 23): 0,
 (0, 24): 0,
 (0, 25): 0,
 (0, 26): 0,
 (0, 27): 0,
 (0, 28): 0,
 (0, 29): 0,
 (0, 30): 0,
 (0, 31): 0,
 (0, 32): 0,
 (0, 33): 0,
 (0, 34): 0,
 (0, 35): 0,
 (0, 36): 0,
 (0, 37): 0,
 (0, 38): 0,
 (0, 39): 0,
 (0, 40): 0,
 (0, 41): 0,
 (0, 42): 0,
 (0, 43): 0,
 (0, 44): 0,
 (0, 45): 0,
 (0, 46): 0,
 (0, 47): 0,
 (0, 48): 0,
 (0, 49): 0,
 (0, 50): 0,
 (0, 51): 0,
 (0, 52): 0,
 (0, 53): 0,
 (0, 54): 0,
 (0, 55): 0,
 (0, 56): 0,
 (0, 57): 0,
 (0, 58): 0,
 (0, 59): 0,
 (0, 60): 0,
 (0, 61): 0,
 (0, 62): 0,
 (0, 63): 0,
 (0, 64): 0,
 (0, 65): 0,
 (0, 66): 0,
 (0, 67): 0,
 (0, 68): 0,
 (0, 69): 0,
 (0, 70): 0,
 (0, 71): 0,
 (0, 72): 0,
 (0, 73): 0,
 (0, 74): 0,
 (0, 75): 0,
 (0, 76): 0,
 (0, 77):

In [16]:
#V is ferry flow network nodes set, U is passengers flow network nodes set
# 定义虚拟节点，其格式和其他节点一致
for i in P:
    # 先求在虚拟节点的客户净需求量
    virtual_demand=-data_demand[data_demand['cus_port']==(i+1)]['volume'].sum()
    D[i][(i,q)]=virtual_demand
    for t in P:
        if t!=i:
            D[i][(t,q)]=0

In [18]:
U=VV[:]
for i in P:
    U.append((i,q))

#### 弧定义 

In [19]:
def ceil(x):
    if int(x)==x:
        return int(x)
    else:
        return int(x)+1

In [20]:
# 先获取每条船可以往返的港口+单程时间
data_time=pd.read_excel(f,sheet_name='Sheet4-time')
# 排除不能到达的港口
data_time=data_time[data_time['tour_time']!=999]

In [21]:
# 定义轮船的arc

In [22]:
ferry_type=['small','small','large'] # 通过船的类型确定往返港口和单程时间
E=[[] for f in F] # 定义每条轮船的edge集合
for f in F: # 对每条船都开始生成可行的弧
    temp_data=data_time[data_time['type']==ferry_type[f]] # 生成每种船对应的data
    for index,row in temp_data.iterrows():
        from_port, to_port, tour_time=row[1:]
        for i in Q:
            for j in range(i+1,q):                
                if from_port==to_port and j>i+1: # 生成同港口的弧
                    break
                try:
                    tour=max(1,ceil(float(tour_time/10)))
                except:
                    continue
                if i+tour!=j: # 从from_port(i)到to_port(j)是否可行的判断  
                    continue
                E[f].append((V[from_port-1][i],V[to_port-1][j]))

In [29]:
# 定义客户流的destination arc还有infeasible arc
Dest=[[] for k in P]
for k in P:
    for i in Q:
        Dest[k].append((V[k][i],(k,q)))

In [30]:
# 定义客户流的infeasible arc
Infeas=[]
for k in P:
    for h in P:
        if k==h:
            continue
        Infeas.append((V[k][q-1],(k,q)))

In [31]:
#define A_k= E and Infeas[k] and Dest[k]
A=[[] for k in P]
EE=[]
for f in F:
    for a in E[f]:
        EE.append(a)
Euni=list(set(EE))
for k in P:
    Atemp=[]
    Atemp=Euni+Infeas+Dest[k]
    A[k]=list(set(Atemp))

#### 成本相关定义

In [32]:
fer_types=set(data_ferry['f'].iloc[:3])
cost={}
for fer_type in fer_types:
    temp_data=data_ferry[data_ferry['f']==fer_type]
    cost[fer_type]={'inport':float(temp_data['gf_inport']),'enroute':float(temp_data['gf_enroute'])}

In [33]:
cost

{'small2': {'inport': 160.0, 'enroute': 400.0},
 'large': {'inport': 240.0, 'enroute': 600.0},
 'small1': {'inport': 160.0, 'enroute': 400.0}}

In [34]:
# 定义每条船在每条弧上的成本
new_ferry_type=['small1','small2','large']
g=[{} for f in F]
for f in F:
    for e in E[f]:
        node1, node2=e
        if node1[0]==node2[0]:
            g[f][e]=cost[new_ferry_type[f]]['inport']*(node2[1]-node1[1])/6
        else:
            g[f][e]=cost[new_ferry_type[f]]['enroute']*(node2[1]-node1[1])/6

In [35]:
# 定义客户流在每条弧上的成本
M=10e8 # 定义大M成本
C=[{} for k in P]
for k in P:
    for f in F:
        for e in E[f]:
            node1, node2=e
            C[k][e]=(node2[1]-node1[1])*10
        for h in Dest[k]:
            C[k][e]=0
        for row in Infeas:
            for e in row:
                C[k][e]=M

#### 决策变量定义

\begin{aligned}
&y_{u v}^{f} \in\{0,1\} \text { for }(u, v) \in E^{f}, f \in F\\
\end{aligned}

In [36]:
# 定义模型
mdl = Model('FSP')
# 定义y
y_sub=[(f,e) for f in F for e in E[f]] # 定义下标
y=mdl.binary_var_dict(y_sub,name='y') # 生成变量

\begin{aligned}
&x_{u v}^{k_i,t} \text { is a nonnegative integer } \operatorname{for}(u, v) \in A^{k_i}, k_i \in V
\end{aligned}

In [ ]:
# 定义x
x_sub=set()
for k in P:
    for t in P:
        if k==t:
            continue
        for i in Q:
            if i==q-1:
                continue
            for f in F:
                for e in E[f]:
                    x_sub.add((V[t][i],k,e))
            for h in P:
                for e in Infeas[h]:
                    x_sub.add((V[t][i],k,e))
            for e in Dest[k]:
                x_sub.add((V[t][i],k,e))
x_list = list(x_sub)
x=mdl.integer_var_dict(x_list,lb=0,name='x')

\begin{aligned}
z_{p_j}^{k_i,p}=\left\{ \begin{array}{ll}
1 & \textrm{至少存在一个客户在$p_j$结点到达目的地$p$} \\
0 & \textrm{otherwise}
\end{array} \right.
\end{aligned}

In [37]:
# 定义z
z_sub=set()
for k in P:
    for t in P:
        if k==t:
            continue
        for i in Q:
            if i==q-1:
                continue
            for j in range(i+1,q):
                z_sub.add((V[t][i],k,V[k][j]))
z_list = list(z_sub)
z=mdl.binary_var_dict(z_list,name='z')

### 目标函数

\begin{aligned}
\text{Minimize} \quad  & \lambda \sum_{f\in F} \sum_{e\in E^f} g_e^f y_e^f+{\color{red} { v \sum_{k \in P}\sum_{e\in A^{k}}c_e^{k} \sum_{v_i \in V,i\ne q }x_e^{v_i,k} }} \\ 
\end{aligned}


In [38]:
mdl.minimize(lam*mdl.sum(g[f][e]*y[f,e] for f in F for e in E[f]) 
            +niu*mdl.sum(C[k][a]*x[V[t][i],k,a] for k in P for t in P  if i not in [114]  for a in A[k] ) )

### 约束

1. 轮船流入流出约束
\begin{aligned}
\sum_{v^{\prime} \in I^{f}(v)} y_{v^{\prime} v}^{f}-\sum_{v^{\prime} \in O^{f}(v)} y_{v v^{\prime}}^{f}=b_{v}^{f} \text { for every } v \in V \text { and } f \in F\\
\end{aligned}



In [39]:
I=[{}for f in F]
O=[{}for f in F]
for f in F:
    for v in VV:
        I[f][v]=[]
        O[f][v]=[]
        for t in E[f]:
            if t[1]==v:
                I[f][v].append(t[0])
            if t[0]==v:
                O[f][v].append(t[1])   

In [40]:
b=[{}for f in F]
for f in F:
    for v in VV:
        hf=data_ferry['hf'][f]-1
        if v==(hf,0):
            b[f][v]=-1
        elif v==(hf,q-1):
            b[f][v]=1
        else:
            b[f][v]=0

In [41]:
mdl.add_constraints(
    mdl.sum(y[f,(v1,v)] for v1 in I[f][v]) 
    - mdl.sum(y[f,(v,v2)] for v2 in O[f][v])
    == b[f][v]
    for v in VV for f in F) 

[docplex.mp.LinearConstraint[](-y_0_((0, 0), (0, 1))-y_0_((0, 0), (1, 4))-y_0_((0, 0), (2, 5))-y_0_((0, 0), (3, 4)),EQ,-1),
 docplex.mp.LinearConstraint[](-y_1_((0, 0), (0, 1))-y_1_((0, 0), (1, 4))-y_1_((0, 0), (2, 5))-y_1_((0, 0), (3, 4)),EQ,0),
 docplex.mp.LinearConstraint[](-y_2_((0, 0), (0, 1))-y_2_((0, 0), (1, 4))-y_2_((0, 0), (2, 5))-y_2_((0, 0), (3, 4)),EQ,0),
 docplex.mp.LinearConstraint[](y_0_((0, 0), (0, 1))-y_0_((0, 1), (0, 2))-y_0_((0, 1), (1, 5))-y_0_((0, 1), (2, 6))-y_0_((0, 1), (3, 5)),EQ,0),
 docplex.mp.LinearConstraint[](y_1_((0, 0), (0, 1))-y_1_((0, 1), (0, 2))-y_1_((0, 1), (1, 5))-y_1_((0, 1), (2, 6))-y_1_((0, 1), (3, 5)),EQ,0),
 docplex.mp.LinearConstraint[](y_2_((0, 0), (0, 1))-y_2_((0, 1), (0, 2))-y_2_((0, 1), (1, 5))-y_2_((0, 1), (2, 6))-y_2_((0, 1), (3, 5)),EQ,0),
 docplex.mp.LinearConstraint[](y_0_((0, 1), (0, 2))-y_0_((0, 2), (0, 3))-y_0_((0, 2), (1, 6))-y_0_((0, 2), (2, 7))-y_0_((0, 2), (3, 6)),EQ,0),
 docplex.mp.LinearConstraint[](y_1_((0, 1), (0, 2))-y_1_((

2. 轮船必须停留一段时间的约束
 \begin{aligned}
&\left|\Delta_{k_{i}}^{f}\right| \sum_{v^{\prime} \in I_{\text {serv }}^{f}\left(k_{i}\right)} y_{v^{\prime} k_{i}}^{f} \leq \sum_{(u, v) \in \Delta_{k_{i}}^{f}} y_{u v}^{f} \text { for every } k_{i} \in V \text { and } f \in F,\\
 \end{aligned}
 
 $\Delta=2$



In [42]:
Iserv=[{} for f in F]
for f in F:
    for v in VV:
        port=v[0]
        Iserv[f][v]=I[f][v]
        for t in Iserv[f][v]:
            if t[0]==port:
                Iserv[f][v].remove(t)

In [43]:
delta=[{} for f in F]
for f in F:
    for v in VV:
        delta[f][v]=[]
        time=v[1]
        port=v[0]
        if time+1<=q-1:
            delta[f][v].append(((port,time),(port,time+1)))
        if time+2<=q-1:
            delta[f][v].append(((port,time+1),(port,time+2)))

In [44]:
mdl.add_constraints(
    2*mdl.sum(y[f,(v1,ki)] for v1 in Iserv[f][ki]) 
    <= mdl.sum(y[f,a] for a in delta[f][ki])
    for ki in VV for f in F) 

[docplex.mp.LinearConstraint[](0,LE,y_0_((0, 0), (0, 1))+y_0_((0, 1), (0, 2))),
 docplex.mp.LinearConstraint[](0,LE,y_1_((0, 0), (0, 1))+y_1_((0, 1), (0, 2))),
 docplex.mp.LinearConstraint[](0,LE,y_2_((0, 0), (0, 1))+y_2_((0, 1), (0, 2))),
 docplex.mp.LinearConstraint[](0,LE,y_0_((0, 1), (0, 2))+y_0_((0, 2), (0, 3))),
 docplex.mp.LinearConstraint[](0,LE,y_1_((0, 1), (0, 2))+y_1_((0, 2), (0, 3))),
 docplex.mp.LinearConstraint[](0,LE,y_2_((0, 1), (0, 2))+y_2_((0, 2), (0, 3))),
 docplex.mp.LinearConstraint[](0,LE,y_0_((0, 2), (0, 3))+y_0_((0, 3), (0, 4))),
 docplex.mp.LinearConstraint[](0,LE,y_1_((0, 2), (0, 3))+y_1_((0, 3), (0, 4))),
 docplex.mp.LinearConstraint[](0,LE,y_2_((0, 2), (0, 3))+y_2_((0, 3), (0, 4))),
 docplex.mp.LinearConstraint[](0,LE,y_0_((0, 3), (0, 4))+y_0_((0, 4), (0, 5))),
 docplex.mp.LinearConstraint[](0,LE,y_1_((0, 3), (0, 4))+y_1_((0, 4), (0, 5))),
 docplex.mp.LinearConstraint[](0,LE,y_2_((0, 3), (0, 4))+y_2_((0, 4), (0, 5))),
 docplex.mp.LinearConstraint[](2y_0_((1,

3. 同一个港口最多停留的轮船数约束
\begin{aligned}
\sum_{f \in F} y_{k_{i} k_{i+1}}^{f} \leq \beta_{k} \text { for every waiting arc }\left(k_{i}, k_{i+1}\right), k_{i} \in V_{k}, k \in P, i \neq q\\
\end{aligned}

In [45]:
beta=[2,2,2,2,1]

In [46]:
mdl.add_constraints(
    mdl.sum(y[f,(V[k][i],V[k][i+1])] for f in F) 
    <= beta[k] for i in range(0,q-1) for k in P  ) 

[docplex.mp.LinearConstraint[](y_0_((0, 0), (0, 1))+y_1_((0, 0), (0, 1))+y_2_((0, 0), (0, 1)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((1, 0), (1, 1))+y_1_((1, 0), (1, 1))+y_2_((1, 0), (1, 1)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((2, 0), (2, 1))+y_1_((2, 0), (2, 1))+y_2_((2, 0), (2, 1)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((3, 0), (3, 1))+y_1_((3, 0), (3, 1))+y_2_((3, 0), (3, 1)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((4, 0), (4, 1))+y_1_((4, 0), (4, 1))+y_2_((4, 0), (4, 1)),LE,1),
 docplex.mp.LinearConstraint[](y_0_((0, 1), (0, 2))+y_1_((0, 1), (0, 2))+y_2_((0, 1), (0, 2)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((1, 1), (1, 2))+y_1_((1, 1), (1, 2))+y_2_((1, 1), (1, 2)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((2, 1), (2, 2))+y_1_((2, 1), (2, 2))+y_2_((2, 1), (2, 2)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((3, 1), (3, 2))+y_1_((3, 1), (3, 2))+y_2_((3, 1), (3, 2)),LE,2),
 docplex.mp.LinearConstraint[](y_0_((4, 1), (4, 2))+y_1_((4, 1), (4, 2))+y_2_((4, 1), (4, 2

4. 客户的流入流出约束
\begin{aligned}
\sum_{v^{\prime} \in I^{k_i,t}_{v
} } x_{v^{\prime}, v}^{k_i,t}-\sum_{v^{\prime} \in O_{v}^{k_i,t}} x_{v, v^{\prime}}^{k_i,t}=-d_{v}^{k_i,t} \text { for every v } \in U^{k_i,t} \text { and for every } (k_i,t) \in (V\times P)\\
\end{aligned}
$I^{k_i,t}_{v }$ 用$I^{t}_{v }$就可以了，同理$O$,$U$

### 下面这个有问题

In [47]:
d=[[{}for ki in VV]for t in P]
for t in P:
    for ki in VV:
        for v in U:
            if v==ki:
                d[t][ki][v]=D[t][ki]
            elif v==(t,q):
                d[t][ki][v]=-D[t][ki]
            else:
                d[t][ki][v]=0

TypeError: list indices must be integers or slices, not tuple

In [48]:
II=[{}for t in P]
OO=[{}for t in P]
for t in P:
    for v in UU:
        II[t][v]=[] #不知道这里为什么出错
        OO[t][v]=[]
        for i in A[t]:
            if i[1]==v:
                II[t][v].append(i[0])
            if i[0]==v:
                OO[t][v].append(i[1])

NameError: name 'UU' is not defined

In [ ]:
# v^prime: u     v:v      ki:a      t:t
mdl.add_constraints(
mdl.sum(x[a,t,(u,v)] for u in II[t][v])-mdl.sum(x[a,t,(v,u)] for u in OO[t][v])== -d[t][a][v]  for v in U for a in V for t in P )

5. 客户流量<=轮船容量约束
\begin{aligned}
\sum_{(k_i,t) \in V\times P} x_{u v}^{k_i,t} \leq \sum_{f \in F(u, v)} C^{f} y_{u v}^{f} \text { for every service } \operatorname{arc}(u, v) \in \Omega\\
\end{aligned}

In [49]:
FF={}
AA=Euni+Infeas
for k in P:
    for i in Dest[k]:
        AA.append(i)
for arc in AA:
    FF[arc]=[]
    for f in F:
        if arc in E[f]:
            FF[arc].append(f)

In [ ]:
mdl.add_constraints(
mdl.sum(x[a,t,(u,v)] for a in V for t in P)<= mdl.sum(y[f,(u,v)]*g[f][(u,v)] for f in F[(u,v)])  for (u,v) in AA )

6. 客户转移时间
\begin{aligned}
\color{red}{
 \sum_{j=i}^{t} \sum_{v^{\prime} \in I_{\text {serv }}\left(k_{j}\right)} x_{v^{\prime} k_{j}}^{k_i,p} \leq x_{k_{t} k_{t+1}}^{k_i,p} \text { for } t=i \text { to } r-1 ,k_{i} \in V, i \neq q \text { and } p \in P \backslash\{k\} }
\end{aligned}

只停留一期
\begin{aligned}
\color{red}{ \sum_{v^{\prime} \in I_{\text {serv }}\left(k_{i}\right)} x_{v^{\prime} k_{i}}^{k_i,p} \leq x_{k_{i} k_{i+1}}^{k_i,p} , k_{i} \in V, i \neq q \text { and } p \in P \backslash\{k\} }
\end{aligned}

In [51]:
I_nof_serv={}
for v in VV:
    I_nof_serv[v]=[]
    for f in F:
        if v in Iserv[f].keys():
            for i in Iserv[f][v]:
                I_nof_serv[v].append(i)

In [ ]:
mdl.add_constraints(
mdl.sum(x[V[k][i],p,(v,V[k][i])] for v in I_nof_serv[(k,i)]<=x[V[k][i],p,(V[k][i],V[k][i+1])]  
        for k in P if i not in [114] for p in P )

7. 硬时间窗
$$\sum_{v^{\prime}\in I_{serve}(p_j)}x_{v^{\prime},p_j}^{k_i,p}\le M z_{p_j}^{k_i,p}  \qquad (1)$$

$$\tau(p_j)\le \tau (a(k_i,p))+M(1-z_{p_j}^{k_i,p}  )\qquad (2)$$

$$ j\le a(k_i,p)+M(1-z_{p_j}^{k_i,p}  )\qquad (2)$$

$$(1),(2) \quad \forall k_i\in V,i\ne q,k\ne p,\forall p\in P, {\color{red}{ \forall j\in (i,q] }}$$

In [52]:
M=10^5

In [ ]:
mdl.add_constraints(
mdl.sum(x[V[k][i],p,(v,V[p][j])] for v in I_nof_serv[(p,j)]<=M*z[V[k][i],p,V[p][j]]  
        for k in P if i not in [114] for p in P for j in range(i+1,q) )

In [ ]:
mdl.add_constraints(
mdl.sum( j<=arri_time[p][(k,i)]+M*(1-z[V[k][i],p,V[p][j]])  
        for k in P if i not in [114] for p in P for j in range(i+1,q) )